In [13]:
import pandas as pd
import pyproj
import numpy as np

# Define the custom Lambert Conformal Conic projection
proj_string = """
    +proj=lcc +datum=NAD83 +ellps=GRS80 +lon_0=-100 +lat_1=27.4166666666667
    +lat_2=34.9166666666667 +lat_0=31.1666666666667 +x_0=999999.9998984
    +y_0=999999.9998984 +unit=us-ft +no_defs
"""

# Initialize the Transformer object
transformer = pyproj.Transformer.from_proj(
    pyproj.Proj(proj_string),  # Source coordinate system
    pyproj.Proj(proj='latlong', datum='WGS84'),  # Destination coordinate system
    always_xy=True  
)

# Load the CSV file in chunks
csv_file_path = 'Updated_Pressure_Data.csv' 

# Set up a CSV writer to write the output to a new file
output_csv_path = 'Updated_Pressure_Data_with_LatLon.csv'
chunksize = 10000  # Process 10,000 rows at a time

# Write the header first
first_chunk = True

for chunk in pd.read_csv(csv_file_path, chunksize=chunksize):
    # Convert from US feet to meters
    chunk['X_meters'] = chunk['X'] * 0.3048
    chunk['Y_meters'] = chunk['Y'] * 0.3048

    # Transform to lat, lon
    chunk['Longitude'], chunk['Latitude'] = transformer.transform(chunk['X_meters'].values, chunk['Y_meters'].values)  

    # Drop the intermediate columns
    chunk.drop(['X_meters', 'Y_meters', 'X', 'Y'], axis=1, inplace=True)

    # Write the chunk to a CSV
    mode = 'w' if first_chunk else 'a'
    header = first_chunk  # Write header only in the first chunk
    chunk.to_csv(output_csv_path, mode=mode, index=False, header=header)
    first_chunk = False

# Return the path to the new CSV file
output_csv_path

'Updated_Pressure_Data_with_LatLon.csv'